**Recuerde no agregar o quitar celdas en este notebook, ni modificar su tipo. Si lo hace, el sistema automaticamente lo calificará con cero punto cero (0.0)**

Obtenga los 5 registros con valores más pequeños en la tercera columna.

In [1]:
%%writefile input.txt
B   1999-08-28   14
E   1999-12-06   121
E   1993-07-21   17
C   1991-02-12   2
E   1995-04-25   161
A   1992-08-22   14
B   1999-06-11   12
E   1993-01-27   8
E   1999-09-10   11
E   1990-05-03   16
E   1994-02-14   101
A   1988-04-27   9
A   1990-10-06   10
E   1985-02-12   16
E   1998-09-14   7
B   1994-08-30   17
A   1997-12-15   13
B   1995-08-23   101
B   1998-11-22   13
B   1997-04-09   6
E   1993-12-27   181
E   1999-01-14   15
A   1992-09-19   18
B   1993-03-02   14
B   1999-10-21   131
A   1990-08-31   12
C   1994-01-25   10
E   1990-02-09   18
A   1990-09-26   5
A   1993-05-08   16
B   1995-09-06   141
E   1991-02-18   14
A   1993-01-11   141
A   1990-07-22   4
C   1994-09-09   151
C   1994-07-27   1
D   1990-10-10   151
A   1990-09-05   11
B   1991-10-01   151
A   1994-10-25   13

Writing input.txt


## Mapper

In [2]:
%%writefile mapper.py
#! /usr/bin/env python

import sys

class Mapper:
    
    NOTHING      = 0
    JUST_COUNTER = 1
    JUST_STATUS  = 2
    ALL          = 3
    
    def __init__(s, log_level=NOTHING):
        s.log_level = log_level
    
    def emit(self, key, value=1):
        sys.stdout.write('{}\t{}\n'.format(key, value))
    
    def status(s, message):
        sys.stderr.write('reporter:status: {}\n'.format(message))
    
    def counter(s, counter, amount=1, group='ApplicationCounter'):
        sys.stderr.write('reporter:counter: {},{},{}\n'.format(group, counter, amount))
    
    def map(s):
        
        counter = 0
        
        for line in s:
            
            counter += 1
            if s.log_level in [2, 3]: s.status('Processing line {}'.format(line))
            if s.log_level in [1, 3]: s.counter('NumberLines', counter)
            
            s.emit('{:0>4}'.format(line[17:]),line)

    def __iter__(s):
        for line in sys.stdin:
            yield line[:-1]
    
if __name__ == "__main__": 
    
    mapper = Mapper()
    mapper.map()

Writing mapper.py


## Reducer

In [3]:
%%writefile reducer.py
#!/usr/bin/env python

import sys
from itertools import groupby

class Reducer:
    
    def __init__(s, str_in, str_out, str_err):
        s.str_in  = str_in
        s.str_out = str_out
        s.str_err = str_err
    
    def emit(s, value):
        s.str_out.write('{}\n'.format(value))
    
    def reduce(s):
        counter=0
        for key, value in s:
            counter+=1
            s.emit(value)
            if counter >= 5: break
    
    def __iter__(s):
        for line in s.str_in:
            key, val = line.split('\t')
            val = val[:-1]
            yield(key,val)

if __name__ == '__main__': 
    reducer = Reducer(sys.stdin,sys.stdout,sys.stderr)
    reducer.reduce()

Writing reducer.py


## Ejecución

In [4]:
%%bash
rm -rf output
STREAM=$HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar
chmod +x mapper.py
chmod +x reducer.py
hadoop jar $STREAM -input input.txt -output output  -mapper mapper.py -reducer reducer.py
cat output/part-00000

C   1994-07-27   1	
C   1991-02-12   2	
A   1990-07-22   4	
A   1990-09-26   5	
B   1997-04-09   6	


In [5]:
!rm -rf mapper.py reducer.py output input.txt

---

Para realizar la evaluación automática de este libro:

* Abra un Terminal.
* Asegurece que esat en la misma carpeta que contiene este notebook.
* Salve el notebook.
* Ejecute el siguiente comando:

      ./gradetool 10-Taller.ipynb

---